In [47]:
import os
from dotenv import load_dotenv
load_dotenv()



True

In [48]:
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAI
from langchain.chains import create_retrieval_chain


In [49]:
loader = PyPDFLoader('attention.pdf')
documents = loader.load()
documents[1]

Document(page_content='1 Introduction\nRecurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 35,2,5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [38, 24, 15].\nRecurrent models typically factor computation along the symbol positions of the input and output\nsequences. Aligning the positions to steps in computation time, they generate a sequence of hidden\nstates ht, as a function of the previous hidden state ht−1and the input for position t. This inherently\nsequential nature precludes parallelization within training examples, which becomes critical at longer\nsequence lengths, as memory constraints limit batching across examples. Recent work has achieved\nsignificant improvements in compu

In [50]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 1000)
chunks = splitter.split_documents(documents)
chunks[1]

Document(page_content='entirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\nto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new single-model state-of-the-art BLEU score of 41.8 after\ntraining for 3.5 days on eight GPUs, a small fraction of the training costs of the\nbest models from the literature. We show that the Transformer generalizes well to\nother tasks by applying it successfully to English constituency parsing both with\nlarge and limited training data.\n∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started\nthe effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer mo

In [51]:
#embedding
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks,embeddings)

In [52]:
prompt =  ChatPromptTemplate.from_template(''' you are an ai resercher , please answer the questions ?
                                           <contex>
                                           {context}
                                           <context/>
                                           Question: {input}''')
llm = OpenAI()


In [56]:
#chains and db retriver
chain = create_stuff_documents_chain(llm,prompt)
retriver = db.as_retriever()
retrieval_chain=create_retrieval_chain(retriver,chain)

In [54]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [55]:
response['answer']

' vs Multi-Head Attention: What are the differences and why do they matter?\n\nScaled Dot-Product Attention and Multi-Head Attention are both types of attention mechanisms used in the Transformer model. The main difference between them is how they compute the weights assigned to the values. In Scaled Dot-Product Attention, the dot products of the query with all keys are divided by the square root of the dimension of the keys, and then a softmax function is applied to obtain the weights. In Multi-Head Attention, the queries, keys, and values are first linearly projected to different dimensions and then the attention function is applied in parallel on each of these projected versions. This allows the model to jointly attend to different representation subspaces at different positions.\n\nThe choice between these two types of attention is important because it affects the performance and computational cost of the model. Scaled Dot-Product Attention is faster and more space-efficient, but m